In [ ]:
import numpy as np
import pandas as pd 

#Machine learning/Modelling Library
import lightgbm as lgb 
from catboost import CatBoostClassifier
import xgboost as xgb

#For splitting
from sklearn.model_selection import KFold,StratifiedKFold, train_test_split 
# for encoding categorical variables

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
#To ignore unnecessary warnings

import warnings
warnings.filterwarnings('ignore') 

%matplotlib inline

In [ ]:
#import dataset
train  = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
#Check for columns in train set
train.columns

In [ ]:
#Print the shape of the train set
print('The Size of the test set is:',test.shape)
#Read the train set
test.head(2)

In [ ]:
train.fillna(-999, inplace=True)
test.fillna(-999, inplace=True)

In [ ]:
cat_features = ['form_field47','default_status']
encoder = LabelEncoder()
encoded = train[cat_features].apply(encoder.fit_transform)

In [ ]:
train = train[['Applicant_ID', 'form_field1', 'form_field2', 'form_field3',
       'form_field4', 'form_field5', 'form_field6', 'form_field7',
       'form_field8', 'form_field9', 'form_field10', 'form_field11',
       'form_field12', 'form_field13', 'form_field14', 'form_field15',
       'form_field16', 'form_field17', 'form_field18', 'form_field19',
       'form_field20', 'form_field21', 'form_field22', 'form_field23',
       'form_field24', 'form_field25', 'form_field26', 'form_field27',
       'form_field28', 'form_field29', 'form_field30', 'form_field31',
       'form_field32', 'form_field33', 'form_field34', 'form_field35',
       'form_field36', 'form_field37', 'form_field38', 'form_field39',
       'form_field40', 'form_field41', 'form_field42', 'form_field43',
       'form_field44', 'form_field45', 'form_field46', 
       'form_field48', 'form_field49', 'form_field50']].join(encoded)
train.head()

In [ ]:
cat_features = ['form_field47']
encoder = LabelEncoder()

encoded = test[cat_features].apply(encoder.fit_transform)

In [ ]:
test = test[['Applicant_ID', 'form_field1', 'form_field2', 'form_field3',
       'form_field4', 'form_field5', 'form_field6', 'form_field7',
       'form_field8', 'form_field9', 'form_field10', 'form_field11',
       'form_field12', 'form_field13', 'form_field14', 'form_field15',
       'form_field16', 'form_field17', 'form_field18', 'form_field19',
       'form_field20', 'form_field21', 'form_field22', 'form_field23',
       'form_field24', 'form_field25', 'form_field26', 'form_field27',
       'form_field28', 'form_field29', 'form_field30', 'form_field31',
       'form_field32', 'form_field33', 'form_field34', 'form_field35',
       'form_field36', 'form_field37', 'form_field38', 'form_field39',
       'form_field40', 'form_field41', 'form_field42', 'form_field43',
       'form_field44', 'form_field45', 'form_field46', 
       'form_field48', 'form_field49', 'form_field50']].join(encoded)
test.head()

In [ ]:
def crossValPred(model):
    avg = 0
    test_scores = []
    for fold, (tr, te) in enumerate(skfolds.split(X, y), 1):
        train_x, test_x = X.iloc[tr], X.iloc[te]
        train_y, test_y = y.iloc[tr], y.iloc[te]
        
        model.fit(train_x, train_y, eval_set=[(train_x, train_y), (test_x, test_y)], 
                  early_stopping_rounds=200, verbose=False)
        
        preds = model.predict_proba(test_x)[:, 1]
        auc = roc_auc_score(test_y, preds)
        print(f'Fold {fold}: Logloss: {auc}')
        avg += auc
        
        
        test_pred = model.predict_proba(test)[:,1]
        test_scores.append(test_pred)
    print(f'Avg Logloss: {avg/fold}')
    return np.mean(test_scores, 0)

In [ ]:
skfolds= StratifiedKFold(n_splits=10, random_state=101, shuffle=True)

In [ ]:
#Defining X and Y variable
X = train.drop(["default_status"], axis =1)
y  = train.default_status

In [ ]:
#Split into train and test set
xtrainval, xtest, ytrainval, ytest = train_test_split(X, y, test_size=0.18, stratify=y)
xtrain, xval, ytrain, yval = train_test_split(xtrainval, ytrainval, test_size=0.25, stratify=ytrainval)

In [ ]:
#Creating model 
cat=CatBoostClassifier(n_estimators=5000, learning_rate = 0.041, eval_metric='AUC', random_seed=10, use_best_model=True)

#Training Catboost Model on train set
cat.fit(xtrain,ytrain,eval_set=[(xval,yval)], early_stopping_rounds=200, verbose=200)

In [ ]:
#Creating model 
cat2=CatBoostClassifier(n_estimators=5000, max_depth=9, learning_rate = 0.01, eval_metric='AUC', reg_lambda = 370)


#Training Catboost Model on train set
cat2.fit(xtrain,ytrain,eval_set=[(xval,yval)], early_stopping_rounds=200, verbose=200)

In [ ]:
cat_pred2 = crossValPred(cat2)

In [ ]:
lgbm = lgb.LGBMClassifier(objective = 'binary',
                       metric = 'auc',
                       boosting_type = 'gbdt',
                       feature_pre_filter = False,
                       lambda_l1 = 0.0,
                       lambda_l2 = 0.0,
                       num_leaves = 5,
                       feature_fraction = 0.6799999999999999,
                       bagging_fraction = 0.9650890426025689,
                       bagging_freq = 1,mini_child_samples =20)

In [ ]:
lgbm.fit(xtrain, ytrain, eval_set=[(xtrain, ytrain), (xval,yval)], verbose=200, early_stopping_rounds=200)

In [ ]:
lgb_pred = crossValPred(lgbm)

In [ ]:
lgbm2 = lgb.LGBMClassifier(objective = 'binary',
                       metric = 'auc',
                       boosting_type = 'gbdt',
                       feature_pre_filter = False,
                       lambda_l1 = 0.0,
                       lambda_l2 = 0.0,
                       num_leaves = 5,
                       feature_fraction = 0.6799999999999999,
                       bagging_fraction = 0.9650890426025689,
                       bagging_freq = 1,
                       min_child_samples = 20)

In [ ]:
lgbm2.fit(xtrain, ytrain, eval_set=[(xtrain, ytrain), (xval,yval)], verbose=200, early_stopping_rounds=200)

In [ ]:
lgb_pred2 = crossValPred(lgbm2)

In [ ]:
xgb = xgb.XGBClassifier(learning_rate=0.01,subsample=0.7,colsample_bytree=0.8,reg_alpha=7,n_estimators=5000,max_depth= 5,random_state=34)

In [ ]:
xgb.fit(xtrain, ytrain, eval_set=[(xtrain, ytrain), (xval,yval)], verbose=200, early_stopping_rounds=200)

In [ ]:
xgb_pred = crossValPred(xgb)

In [ ]:
sample = pd.read_csv('SampleSubmission.csv')

In [ ]:
reblend = ((xgb_pred*0.33) + (cat_pred*0.67))*0.95 + (lgb_pred*0.05)

In [ ]:
reblend = sample.copy()

In [ ]:
reblend.to_csv('XGB_CAT111.csv', index = False)

In [ ]:
blend = (3 * lgb_pred * cat_pred * xgb_pred)/5

In [ ]:
a = pd.read_csv('a.csv')
b = pd.read_csv('b.csv')
c = pd.read_csv('c.csv')

In [ ]:
a.default_status= ((a.default_status*0.33) + (b.default_status*0.67))*0.95 + (c.default_status*0.05)

In [ ]:
a.head()

In [ ]:
a.to_csv('bled6.csv', index = False)